# LLM Chain（LLM链）

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI()
prompt_tpl = PromptTemplate.from_template(
    '请给我讲1个关于{type}的笑话'
)

chain = LLMChain(llm=llm, prompt=prompt_tpl)
print(chain.invoke('程序员'))

In [ ]:
llm = OpenAI()
prompt_tpl = PromptTemplate.from_template(
    '请给我讲1个关于{type}的笑话'
)

chain = LLMChain(
    llm=llm, prompt=prompt_tpl,
    output_key='content' # 默认为 text
)
print(chain.invoke('程序员'))

In [ ]:
llm = OpenAI()
prompt_tpl = PromptTemplate.from_template(
    '请给我讲{count}个关于{type}的笑话'
)

chain = LLMChain(llm=llm, prompt=prompt_tpl)
print(chain.invoke({'count': 2, 'type': '程序员'}))
print(chain.predict(count=1, type='程序员'))

In [ ]:
llm = OpenAI()
prompt_tpl = PromptTemplate.from_template(
    '请给我讲1个关于{type}的笑话'
)

chain = LLMChain(llm=llm, prompt=prompt_tpl)
inputs = [
    {'type': '老师'},
    {'type': '动物'},
    {'type': '植物'},
]
res = chain.apply(inputs)
for info in res:
    print(info)

print('='*20)
# batch 还支持设置最大并发数 max_concurrency
# 如果希望最大并发为5，则可以写为：
# res = chain.batch(inputs, config={"max_concurrency": 5})
res = chain.batch(inputs)
for info in res:
    print(info)

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
instructions = output_parser.get_format_instructions()

llm = OpenAI()
prompt_tpl = PromptTemplate(
    template='请给我举例3个最具有代表性的{type}名称\n{instructions}',
    input_variables=['type'],
    partial_variables={'instructions': instructions}
)

chain = LLMChain(
    llm=llm,
    prompt=prompt_tpl,
    output_parser=output_parser
)
print(chain.invoke('编程语言'))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, load_chain

prompt_tpl = PromptTemplate.from_template(
    '请给我讲1个关于{type}的笑话'
)

llm = OpenAI()
chain = LLMChain(llm=llm, prompt=prompt_tpl)
# 将chain保存到本地，支持 json 和 yaml 格式
chain.save('chain.json')

# 加载本地保存的chain文件
new_chain = load_chain('chain.json')
print(chain.invoke('程序员'))

# Sequential Chain（顺序链）
## SimpleSequentialChain

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

script_prompt_tpl = PromptTemplate.from_template(
    '你是一个优秀的编剧。请使用你丰富的想象力根据我定的标题编写一个故事概要。'
    '标题:{title}'
)

script_llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.9,
    max_tokens=1024
)

script_chain = LLMChain(llm=script_llm, prompt=script_prompt_tpl)

# 创建广告链
adv_prompt_tpl = PromptTemplate.from_template(
    '你是一个优秀的广告写手。请根据我定的故事概要，'
    '为我的故事写一段尽可能简短但要让人有观看欲望的广告词。'
    '故事概要:{story}'
)
adv_llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.6,
    max_tokens=2048
)
adv_chain = LLMChain(llm=adv_llm, prompt=adv_prompt_tpl)

# 将剧本链和广告链串联起来
chain = SimpleSequentialChain(
    chains=[script_chain, adv_chain],
    verbose=True
)
print(chain.invoke('孙悟空大战变型金刚'))

## SequentialChain

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

# 创建剧本链
script_prompt_tpl = PromptTemplate.from_template(
    '你是一个优秀的编剧。请使用你丰富的想象力根据我定的信息编写一个故事概要。'
    '标题:{title}\n故事类型:{story_type}\n主角名:{name}'
)

script_llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.9,
    max_tokens=1024
)

script_chain = LLMChain(
    llm=script_llm,
    prompt=script_prompt_tpl,
    output_key='story'
)

# 创建广告链
adv_prompt_tpl = PromptTemplate.from_template(
    '你是一个优秀的广告词写手。请根据我给定的故事概要，'
    '为我的故事写一段简短但要让人有观看欲望的广告词。'
    '故事标题:{title}\n'
    '故事概要:{story}\n'
    '广告词:'
)
adv_llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.6,
    max_tokens=1024
)
adv_chain = LLMChain(
    llm=adv_llm,
    prompt=adv_prompt_tpl,
    output_key='adv'
)

# 将剧本链和广告链串联起来
chain = SequentialChain(
    chains=[script_chain, adv_chain],
    input_variables=['title', 'story_type', 'name'],
    # 如果希望输出script_chain的结果
    # 这里可以设置为['story', 'adv']
    output_variables=['adv']
)
print(chain.invoke(
    {'title': '上海滩的秘密', 'story_type': '玄幻修真', 'name': '叶豪'}
))

# RouterChain（路由链）

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import (
    LLMRouterChain,
    RouterOutputParser
)
from langchain.chains.router.multi_prompt_prompt import (
    MULTI_PROMPT_ROUTER_TEMPLATE
)

physics_prompt_tpl = PromptTemplate.from_template(
    '你是个优秀的物理学家，你很擅长用简明易懂的方式回答有关物理学的问题。'
    '请使用英文帮我解答下列问题：\n{input}'
)
math_prompt_tpl = PromptTemplate.from_template(
    '你是个很好的数学家。你很擅长回答数学问题。'
    '请使用英文帮我解答下列问题：\n{input}'
)

# 创建模板信息列表
prompt_infos = [
    {
        'name': 'physics',
        'description': '用于解答物理相关问题',
        'prompt_template': physics_prompt_tpl,
    },
    {
        'name': 'math',
        'description': '用于解答数学相关问题',
        'prompt_template': math_prompt_tpl,
    },
]
llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.1,
)

# 生成键为模板名称、值为Chain的字典
destination_chains = {}
for p_info in prompt_infos:
    name = p_info['name']
    prompt = p_info['prompt_template']
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# 将模板名称和模板描述通过MULTI_PROMPT_ROUTER_TEMPLATE生成模板
destinations = [f'{p["name"]}: {p["description"]}'
                for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 这里创建了一个default_chain
# 为了防止提的问题类型并没有包含在prompt_infos中
default_chain = ConversationChain(llm=llm, output_key='text')
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

print(chain.invoke("What is Newton's First Law?"))

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains.router.embedding_router import EmbeddingRouterChain

names_and_descriptions = [
    ('physics', ['用于解答物理相关问题']),
    ('math', ["用于解答数学相关问题"]),
]

router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions,
    Chroma,
    OpenAIEmbeddings(),
    routing_keys=['input']
)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

print(chain.invoke('在物理学科中，牛顿第一定律是什么？'))

# TransformChain（转换链）

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import (
    TransformChain,
    LLMChain,
    SimpleSequentialChain
)


# 用于获取输入的前三段内容
def transform_func(inputs):
    text = inputs['text']
    shortened_text = '\n\n'.join(text.split('\n\n')[:3])
    return {'output_text': shortened_text}


transform_chain = TransformChain(
    input_variables=['text'],
    output_variables=['output_text'],
    transform=transform_func
)

# 用户总结TransformChain发过来的数据
template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(
    input_variables=['output_text'],
    template=template
)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

# 串联transform_chain、llm_chai
sequential_chain = SimpleSequentialChain(
    chains=[transform_chain, llm_chain]
)
with open('story.txt') as f:
    sequential_chain.invoke(f.read())

# SumarizeChain（总结链）

In [ ]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# 加载并分割文档
loader = TextLoader('chat.txt')
spliter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0
)
documents = spliter.split_documents(loader.load())[:5]

llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.2,
)

# 初始化SumarizeChain
chain = load_summarize_chain(
    llm=llm,
    chain_type='stuff',
)
print(chain.invoke(documents))

# APIChain与LLMRequestsChain

## APIChain

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs

llm = OpenAI(temperature=0)
chain = APIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=open_meteo_docs.OPEN_METEO_DOCS,
    limit_to_domains=['https://api.open-meteo.com/'],
)
print(chain.invoke('2024-07-25北京是多少度？'))

## LLMRequestsChain

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMRequestsChain, LLMChain

template = """Between >>> and <<< are the raw search result text from bing.
Extract the answer to the question '{query}' or say 'not found'
if the information is not contained.
Use the format
Extracted:<answer or 'not found'>
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate.from_template(template)
chain = LLMRequestsChain(
    llm_chain=LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT),
)
question = 'On what day will the Hangzhou Asian Games be held?'
inputs = {
    'query': question,
    'url': ('https://www.bing.com/search?form=&q=' +
            question.replace(' ', '+')),
}
print(chain.invoke(inputs))

# SQL Chain（数据库链）

## SQLDatabaseChain

In [ ]:
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

llm = OpenAI()
db = SQLDatabase.from_uri('sqlite:///students.sqlite')
chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

print(chain.invoke("What is the age of xiaoli?"))
print(chain.invoke("What is the phone number of xiaoli's father"))

## SQL Agent

In [ ]:
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase.from_uri('sqlite:///students1.sqlite')
llm = OpenAI()
executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
)

print(executor.invoke('Delete all contents in the info table'))

# QA Chain（问答链）

## ConversationChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

chat = ChatOpenAI()
conversation = ConversationChain(llm=chat)
print(conversation.invoke('你是谁？'))

## RetrievalQA

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


texts = [
    '在这里！',
    '你好啊！',
    '你叫什么名字？',
    '我的朋友们都叫我小明',
    '哦，那太棒了！'
]

# 存储数据
db = Chroma.from_texts(
    texts, OpenAIEmbeddings(),
    persist_directory='./chroma_test/'
)

# 创建问答链
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=db.as_retriever(),
)
print(qa_chain.invoke('你叫什么名字？'))

## ConversationalRetrievalChain

In [ ]:
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

texts = [
    '在这里！',
    '我的年龄是20岁',
    '你叫什么名字？',
    '我的朋友们都叫我小明',
    '哦，那太棒了！'
]

# 存储数据
db = Chroma.from_texts(
    texts, OpenAIEmbeddings(),
    persist_directory='./chroma_test/'
)

# 创建问答链
qa_chain = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    db.as_retriever()
)
chat_history = []
# 开始问答
# chat_history为必传参数，因为默认提示模板会用到
res = qa_chain.invoke({
    "question": '你叫什么名字',
    "chat_history": chat_history
})

# 将返回值存入chat_history，当作下次提问的提示词的一部分传给LLM
chat_history.append((res['question'], res['answer']))

print(qa_chain.invoke({
    "question": '你今年几岁了',
    "chat_history": chat_history
}))

# LangChain Expression Language（LCEL）

## 管道操作符

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

prompt_tpl = PromptTemplate.from_template(
    '请给我讲1个关于{type}的笑话'
)

chain = prompt_tpl | OpenAI()
print(chain.invoke({'type': '程序员'}))

## 在链中设置参数

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import (
    JsonOutputFunctionsParser
)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(
    '请告诉我，我所提的问题属于哪个分类的学科。\n问题：{question}'
)
functions = [
    {
        'name': 'question_category',
        'description': '记录学科的分类',
        'parameters': {
            'type': 'object',
            'properties': {
                'category': {
                    'type': 'string',
                    'description': '问题所对应的学科分类'
                },
                'question': {
                    'type': 'string',
                    'description': '我提问的问题'
                }
            },
            'required': ['category', 'question']
        }
    }
]
chain = prompt | model.bind(
    function_call={'name': 'question_category'},
    functions=functions
) | JsonOutputFunctionsParser()

print(chain.invoke({'question': '牛顿第一定律是什么？'}))

## 配置

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import ConfigurableField

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id='llm_temperature',
        name='LLM Temperature',
        description='LLM的温度设置',
    )
)
prompt = PromptTemplate.from_template(
    '请给我讲一个关于{type}的笑话'
)

chain = prompt | model
res = chain.with_config(
    configurable={'llm_temperature': 0.9}
).invoke({'type': '程序员'})

print(res)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import ConfigurableField

model = ChatOpenAI()

story = PromptTemplate.from_template('请给我讲一个关于{type}的故事')
joke = PromptTemplate.from_template('请给我讲一个关于{type}的冷笑话')

prompt = PromptTemplate.from_template(
    '请给我写一个关于{type}的演讲词'
).configurable_alternatives(
    ConfigurableField(id='prompt'),
    story=story,
    joke=joke,
    # 设置默认模板对应的key
    default_key='speech'
)

chain = prompt | model
res = chain.with_config(
    configurable={'prompt': 'joke'}
).invoke({'type': '程序员'})

print(res)

## 设置备用方案

In [ ]:
from openai import RateLimitError
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

openai_llm = ChatOpenAI(max_retries=0)
anthropic_llm = ChatAnthropic()
llm = openai_llm.with_fallbacks(
    [anthropic_llm],
    exceptions_to_handle=(RateLimitError,)
)
llm.invoke('请给我讲一个笑话')

## 获取输入并运行自定义函数

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('question:{question}')
llm = ChatOpenAI()

chain = (
    {'question': RunnablePassthrough()} | prompt | llm
)
print(chain.invoke('你是谁？'))

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from operator import itemgetter

prompt = ChatPromptTemplate.from_template('question:{question}')
llm = ChatOpenAI()

chain = (
    {'question': itemgetter('question')} | prompt | llm
)
print(chain.invoke({'question': '你是谁？', 'metadata': 'xxx'}))

In [ ]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('question:{question}')
llm = ChatOpenAI()

def print_intput(text):
    print(text)
    return {'question': text}


chain = (RunnableLambda(print_intput) | prompt | llm)
print(chain.invoke('你是谁？'))

In [ ]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('question:{question}')
llm = ChatOpenAI()


def print_intput(text, config):
    print(text)
    print(config['foo'])
    return {'question': text}


chain = (RunnableLambda(print_intput) | prompt | llm)
print(chain.invoke('你是谁？', config={'foo': 'bar'}))

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('question:{question}')
llm = ChatOpenAI()

def handle_metadata(data):
    return {'user': 'xiaoming'}

def handle_question(data):
    return '请问，' + data['question']

chain = (
        RunnablePassthrough.assign(
            metadata=handle_metadata,
            question=handle_question,
        )| RunnablePassthrough() | prompt | llm
)

print(chain.invoke({'question': '你是谁'}))

## 路由链

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

model = ChatOpenAI()
joke_tpl = ChatPromptTemplate.from_template('讲一个关于{type}的笑话')
joke_chain = joke_tpl | model
story_tpl = ChatPromptTemplate.from_template('讲一个关于{type}的故事')
story_chain = story_tpl | model

map_chain = RunnableParallel(joke=joke_chain, poem=story_chain)
print(map_chain.invoke({'type': '动物'}))

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain.chains import create_tagging_chain_pydantic
from langchain.schema.runnable import (
    RouterRunnable, RunnablePassthrough
)


class SelectChain(BaseModel):
    name: str = Field(description='从 "physics" 和 "math" 二选一')

# 创建一个用于确定使用哪个选择链
tagger = create_tagging_chain_pydantic(
    SelectChain, ChatOpenAI(temperature=0)
)

model = ChatOpenAI()

physics_prompt_tpl = PromptTemplate.from_template(
    '你是个优秀的物理学家，你很擅长用简明易懂的方式回答有关物理学的问题。'
    '请使用英文帮我解答下列问题：\n{question}'
)
math_prompt_tpl = PromptTemplate.from_template(
    '你是个很好的数学家。你很擅长回答数学问题。'
    '请使用英文帮我解答下列问题：\n{question}'
)

# 创建一个路由链
router = RouterRunnable({
    'physics': physics_prompt_tpl | model,
    'math': math_prompt_tpl | model
})

chain = {
            'key': RunnablePassthrough() | tagger | (
                lambda x: x['text'].name),
            'input': {'question': RunnablePassthrough()}
        } | router

print(chain.invoke('物理中的牛顿第一定律是怎么定义的？'))